In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Move folder from drive and install requirements


In [ ]:
!unzip /content/drive/MyDrive/HIISS/Causal-Unsupervised-Segmentation.zip &> /dev/null
!mv /content/content/Causal-Unsupervised-Segmentation /content
!rm -r /content/content

In [ ]:
!pip install -r /content/Causal-Unsupervised-Segmentation/requirements.txt &> /dev/null
!pip install timm==0.9.5 &> /dev/null
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git &> /dev/null

# Load pirelli dataset

In [ ]:
!unzip /content/drive/MyDrive/HIISS/datasets/Pirelli_4cat_no_stride.zip -d /content &> /dev/null

In [ ]:
!unzip /content/drive/MyDrive/HIISS/datasets/Pirelli_unsupervised.zip -d /content &> /dev/null

# Crop the dataset


In [ ]:
!python ./Causal-Unsupervised-Segmentation/crop_dataset.py  --data_dir "Pirelli_4cat_no_stride" --dataset "pirelli" --crop_type "super"

Device = cuda:0
100% 130/130 [00:25<00:00,  5.04it/s]


In [ ]:
!python ./Causal-Unsupervised-Segmentation/crop_dataset.py --data_dir "pirelli_unsupervised" --dataset "pirelli" --crop_type "super"

Device = cuda:0
100% 2651/2651 [20:20<00:00,  2.17it/s]


# Train the mediator
Namely get the clusterbook

In [ ]:
!rm -r CAUSE # Remove to retrain again

rm: cannot remove 'CAUSE': No such file or directory


In [ ]:
!python ./Causal-Unsupervised-Segmentation/train_mediator.py \
                          --data_dir "pirelli_unsupervised" --dataset "pirelli" \
                          --ckpt "/content/drive/MyDrive/HIISS/visual.pth"\
                          --gpu 0 --epoch 1 --train_resolution 224 --test_resolution 224  \
                          #--num_codebook 32 --reduced_dim 4 --projection_dim 32

------------------Configurations------------------
epoch: 1
distributed: True
load_segment: False
load_cluster: False
train_resolution: 224
test_resolution: 224
batch_size: 16
num_workers: 0
data_dir: pirelli_unsupervised
dataset: pirelli
ckpt: /content/drive/MyDrive/HIISS/visual.pth
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 768
-------------------------------------------------
2024-01-24 17:35:54.284299: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 17:35:54.284400: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 17:35:54.410883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to registe

# Train the transformer

In [ ]:
!python ./Causal-Unsupervised-Segmentation/train_front_door_tr.py \
          --data_dir "Pirelli_4cat_no_stride" --dataset "pirelli" \
          --ckpt "/content/drive/MyDrive/HIISS/visual.pth" \
          --gpu 0 --epoch 5 --batch_size 16 --train_resolution 224 --test_resolution 224
          #--num_codebook 32 --reduced_dim 4 --projection_dim 32

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat_no_stride
dataset: pirelli
ckpt: /content/drive/MyDrive/HIISS/visual.pth
epoch: 5
distributed: False
load_segment: False
load_cluster: False
train_resolution: 224
test_resolution: 224
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 768
num_queries: 196
-------------------------------------------------
2024-01-24 17:55:24.874194: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 17:55:24.874252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 17:55:24.881512: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to registe

#Fine tuning

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_4cat_no_stride" --dataset "pirelli" \
          --ckpt "/content/drive/MyDrive/HIISS/visual.pth"  \
          --gpu 0 --epoch 3 --batch_size 16 --load_segment True \
          --load_cluster False --train_resolution 224 --test_resolution 224
          #--num_codebook 32 --reduced_dim 4 --projection_dim 32

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat_no_stride
dataset: pirelli
ckpt: /content/drive/MyDrive/HIISS/visual.pth
epoch: 3
distributed: True
load_segment: True
load_cluster: False
train_resolution: 224
test_resolution: 224
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 768
num_queries: 196
-------------------------------------------------
2024-01-24 18:02:51.414380: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 18:02:51.414445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 18:02:51.415933: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register 

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_4cat_no_stride" --dataset "pirelli" \
          --ckpt "/content/drive/MyDrive/HIISS/visual.pth"  \
          --gpu 0 --epoch 1 --batch_size 16 --load_segment True \
          --load_cluster False --train_resolution 224 --test_resolution 224
          #--num_codebook 32 --reduced_dim 4 --projection_dim 32

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat_no_stride
dataset: pirelli
ckpt: /content/drive/MyDrive/HIISS/visual.pth
epoch: 1
distributed: True
load_segment: True
load_cluster: False
train_resolution: 224
test_resolution: 224
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 768
num_queries: 196
-------------------------------------------------
2024-01-24 18:06:38.810460: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 18:06:38.810511: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 18:06:38.811887: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register 

# Test the model

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/test_tr.py \
        --data_dir "Pirelli_4cat_no_stride" --dataset "pirelli" \
        --ckpt "/content/drive/MyDrive/HIISS/visual.pth" \
        --load_segment True --load_cluster True --train_resolution 224 --test_resolution 224
        #--num_codebook 32 --reduced_dim 4 --projection_dim 32

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat_no_stride
dataset: pirelli
port: 12355
ckpt: /content/drive/MyDrive/HIISS/visual.pth
distributed: False
load_segment: True
load_cluster: True
train_resolution: 224
test_resolution: 224
batch_size: 16
num_workers: 0
gpu: 0
num_codebook: 2048
just_imgs: False
reduced_dim: 90
projection_dim: 2048
dim: 768
num_queries: 196
-------------------------------------------------
2024-01-24 18:08:32.443703: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 18:08:32.443770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 18:08:32.445273: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuB

# Save Results and checkpoints

In [ ]:
!zip -r results_visual_4classes.zip ./results &> /dev/null

In [ ]:
!zip -r /content/CAUSE_visual_4classes.zip ./CAUSE &> /dev/null

# Experiments with 3 classes

In [ ]:
!rm -r /content/CAUSE

## Load Pirelli 3 classes dataset

In [ ]:
!unzip /content/drive/MyDrive/HIISS/datasets/Pirelli_3cat_no_stride.zip -d /content &> /dev/null

## Crop dataset

In [ ]:
!python ./Causal-Unsupervised-Segmentation/crop_dataset.py  --data_dir "Pirelli_3cat_no_stride" --dataset "pirelli" --crop_type "super"

Device = cuda:0
100% 130/130 [00:25<00:00,  5.06it/s]


# Train Mediator

In [ ]:
!python ./Causal-Unsupervised-Segmentation/train_mediator.py \
                          --data_dir "pirelli_unsupervised" --dataset "pirelli" \
                          --ckpt "/content/drive/MyDrive/HIISS/visual.pth"\
                          --gpu 0 --epoch 1 --train_resolution 224 --test_resolution 224  \
                          #--num_codebook 32 --reduced_dim 4 --projection_dim 32

------------------Configurations------------------
epoch: 1
distributed: True
load_segment: False
load_cluster: False
train_resolution: 224
test_resolution: 224
batch_size: 16
num_workers: 0
data_dir: pirelli_unsupervised
dataset: pirelli
ckpt: /content/drive/MyDrive/HIISS/visual.pth
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 768
-------------------------------------------------
2024-01-24 18:14:29.829247: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 18:14:29.829309: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 18:14:29.831364: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to registe

# Train Transformer

In [ ]:
!python ./Causal-Unsupervised-Segmentation/train_front_door_tr.py \
          --data_dir "Pirelli_3cat_no_stride" --dataset "pirelli" \
          --ckpt "/content/drive/MyDrive/HIISS/visual.pth" \
          --gpu 0 --epoch 5 --batch_size 16 --train_resolution 224 --test_resolution 224
          #--num_codebook 32 --reduced_dim 4 --projection_dim 32

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat_no_stride
dataset: pirelli
ckpt: /content/drive/MyDrive/HIISS/visual.pth
epoch: 5
distributed: False
load_segment: False
load_cluster: False
train_resolution: 224
test_resolution: 224
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 768
num_queries: 196
-------------------------------------------------
2024-01-24 18:41:42.077460: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 18:41:42.077508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 18:41:42.083011: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to registe

# Fine-tuning

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_3cat_no_stride" --dataset "pirelli" \
          --ckpt "/content/drive/MyDrive/HIISS/visual.pth"  \
          --gpu 0 --epoch 3 --batch_size 16 --load_segment True \
          --load_cluster False --train_resolution 224 --test_resolution 224
          #--num_codebook 32 --reduced_dim 4 --projection_dim 32

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat_no_stride
dataset: pirelli
ckpt: /content/drive/MyDrive/HIISS/visual.pth
epoch: 3
distributed: True
load_segment: True
load_cluster: False
train_resolution: 224
test_resolution: 224
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 768
num_queries: 196
-------------------------------------------------
2024-01-24 18:49:26.296835: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 18:49:26.296893: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 18:49:26.298256: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register 

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_3cat_no_stride" --dataset "pirelli" \
          --ckpt "/content/drive/MyDrive/HIISS/visual.pth"  \
          --gpu 0 --epoch 1 --batch_size 16 --load_segment True \
          --load_cluster False --train_resolution 224 --test_resolution 224
          #--num_codebook 32 --reduced_dim 4 --projection_dim 32

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat_no_stride
dataset: pirelli
ckpt: /content/drive/MyDrive/HIISS/visual.pth
epoch: 1
distributed: True
load_segment: True
load_cluster: False
train_resolution: 224
test_resolution: 224
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 768
num_queries: 196
-------------------------------------------------
2024-01-24 18:53:23.098619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 18:53:23.098684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 18:53:23.100774: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register 

# Test Model

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/test_tr.py \
        --data_dir "Pirelli_3cat_no_stride" --dataset "pirelli" \
        --ckpt "/content/drive/MyDrive/HIISS/visual.pth" \
        --load_segment True --load_cluster True --train_resolution 224 --test_resolution 224
        #--num_codebook 32 --reduced_dim 4 --projection_dim 32

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat_no_stride
dataset: pirelli
port: 12355
ckpt: /content/drive/MyDrive/HIISS/visual.pth
distributed: False
load_segment: True
load_cluster: True
train_resolution: 224
test_resolution: 224
batch_size: 16
num_workers: 0
gpu: 0
num_codebook: 2048
just_imgs: False
reduced_dim: 90
projection_dim: 2048
dim: 768
num_queries: 196
-------------------------------------------------
2024-01-24 18:55:23.547768: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 18:55:23.547826: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 18:55:23.549198: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuB

# Save Results and checkpoints

In [ ]:
!zip -r results_visual_3classes.zip ./results &> /dev/null

In [ ]:
!zip -r /content/CAUSE_visual_3classes.zip ./CAUSE &> /dev/null